# BERT-Klassifikation der Heise-Newsticker-Meldungen

# Torch-Konfiguration

In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU %s" % torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU :-(")

Using GPU Tesla T4


# Daten einlesen

In [2]:
import os
import urllib.request

# File name and URL
file_name = "newsticker-2022-2023-good-bad.csv.gz"
url = "https://github.com/datanizing/m3-llm-workshop/raw/main/newsticker-2022-2023-good-bad.csv.gz"

# Check if the file exists, if not, download it
if not os.path.isfile(file_name):
    print(f"{file_name} does not exist. Downloading...")
    urllib.request.urlretrieve(url, file_name)
    print(f"Downloaded {file_name}.")
else:
    print(f"{file_name} already exists.")

newsticker-2022-2023-good-bad.csv.gz does not exist. Downloading...
Downloaded newsticker-2022-2023-good-bad.csv.gz.


In [3]:
import pandas as pd

df = pd.read_csv("newsticker-2022-2023-good-bad.csv.gz", index_col="id")

In [4]:
df

,headline,comments,time,quality
id,,,,
6233916,Zentrum für KI in der Medizin ​soll für modern...,1.0,2023-11-16 09:20:00.000000,bad
6266836,Bericht: Höchststand bei Anschlägen auf Geldau...,132.0,2022-12-03 17:17:00.000000,good
6287308,Werkstattberichte: Neues aus den Fablabs und d...,2.0,2022-01-18 10:07:00.000000,bad
6315231,Apple wünscht sich mehr Bluetooth-Bandbreite,109.0,2022-01-03 13:38:00.000000,good
6315548,HoloLens Summit: virtuelle Konferenz für Anwen...,1.0,2022-01-01 12:05:00.000000,bad
...,...,...,...,...
9584239,Neue Förderrichtlinie: Bundesregierung drängt ...,557.0,2023-12-31 10:20:00.000000,good
9584333,37C3: Übertreibt es nicht mit der Softwareisie...,98.0,2023-12-31 11:33:00.000000,good
9584447,EU-Vorschlag: Selbstverpflichtung statt Cookie...,100.0,2023-12-31 15:35:00.000000,good


In [5]:
# Labels auf Integer wandeln, pytorch unterstützt nur Integer-Labels
df["label"] = 0
df.loc[df["quality"] == "good", "label"] = 1
df.head()

,headline,comments,time,quality,label
id,,,,,
6233916,Zentrum für KI in der Medizin ​soll für modern...,1.0,2023-11-16 09:20:00.000000,bad,0
6266836,Bericht: Höchststand bei Anschlägen auf Geldau...,132.0,2022-12-03 17:17:00.000000,good,1
6287308,Werkstattberichte: Neues aus den Fablabs und d...,2.0,2022-01-18 10:07:00.000000,bad,0
6315231,Apple wünscht sich mehr Bluetooth-Bandbreite,109.0,2022-01-03 13:38:00.000000,good,1
6315548,HoloLens Summit: virtuelle Konferenz für Anwen...,1.0,2022-01-01 12:05:00.000000,bad,0


In [6]:
# in Arrays wandeln
text = df["headline"].values
labels = df["label"].values

# Tokenisierung

In [7]:
from transformers import AutoTokenizer

# model_name = "google-bert/bert-base-german-cased"
model_name = "dbmdz/bert-base-german-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/247k [00:00<?, ?B/s]

In [8]:
# Bestimmung der Maximallänge der Sätze, um Platz zu sparen
max_len = max([len(tokenizer.encode(t, add_special_tokens=True)) for t in text])
max_len

34

In [10]:
# Jetzt alle Sätze tokenisieren und IDs merken
input_ids = []
attention_masks = []

for t in text:
    encoded_dict = tokenizer.encode_plus(
                        t,
                        add_special_tokens = True,    # '[CLS]' und '[SEP]'
                        max_length = max_len,
                        truncation = True,
                        padding='max_length',
                        return_attention_mask = True,  # Attention-Masks erzeugen
                        return_tensors = 'pt',         # pytorch-Tensoren als Ergebnis
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Python-Listen in Tensoren wandeln
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels) #labels.clone().detach()

# Headline, Tokenisierung und IDs anzeigen
print(text[0])
print(tokenizer.tokenize(text[0]))
print(input_ids[0])

SyntaxError: invalid syntax (<ipython-input-10-b1b4ac522af8>, line 21)

# Daten aufteilen

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Wir arbeiten ab jetzt nur noch mit dem Input-Tensor, der Attention Mask und den Labeln
dataset = TensorDataset(input_ids, attention_masks, labels)

# wir nutzen einen 3:1-Split für Training und Validierung
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
# reproduzierbar arbeiten!
torch.manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(train_size, val_size)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# die BERT-Autoren empfehlen für Finetuning Batch-Sizes von 16 oder 32
batch_size = 32

# DataLoader für die beiden Datensets erzeugen (man könnte auch RandomSampler verwenden)
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)
validation_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

# Modell laden

In [ ]:
from transformers import AutoModelForSequenceClassification

# das Modell muss zum Tokenizer passen!
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels = 2, # wir haben nur gut oder shlecht
    output_attentions = False,
    output_hidden_states = False # wir benötigen keine Embeddings
)
# hier ohne GPU model.cpu() einsetzen
model.cuda()

In [ ]:
# Optimierer auswählen, AdamW ist Standard
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr = 2e-5)

In [ ]:
from transformers import get_linear_schedule_with_warmup

# vier Epochen, das kann justiert werden
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [ ]:
import numpy as np

# Accuracy berechnen
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import random
import numpy as np
from tqdm.auto import trange, tqdm

# alle Zufallszahlengeneratoren initialisieren (Reproduzierbarkeit)
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Statistik für das Training
training_stats = []

for epoch_i in trange(epochs, desc="Epoche"):
    # akkumulierter Loss für diese Epoche
    total_train_loss = 0

    # Modell in Trainingsmodus stellen
    model.train()

    # Trainig pro Batch
    for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        # Daten entpacken und in device-Format wandeln
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Gradienten löschen
        model.zero_grad()

        # Vorwärts-Auswertung (Trainingsdaten vorhersagen)
        res = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)

        # Loss berechnen und akkumulieren
        total_train_loss += res.loss.item()

        # Rückwärts-Auswertung, um Gradienten zu bestimmen
        res.loss.backward()

        # Gradient beschrÃ¤nken wegen Exploding Gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Parameter und Lernrate aktualisieren
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)



    # Modell in Vorhersage-Modus umstellen
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Eine Epoche validieren
    for batch in tqdm(validation_dataloader, desc="Validierung"):
        # jetzt die Validierungs-Daten entpacken
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Rückwärts-Auswertung wird nicht benötigt, daher auch kein Gradient
        with torch.no_grad():
            # Vorhersage durchfÃ¼hren
            res = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        # Loss akkumulieren
        total_eval_loss += res.loss.item()

        # Vorhersagedaten in CPU-Format wandeln, um Accuracy berechnen zu können
        logits = res.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Accuracy für die Verifikation
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    tqdm.write("Accuracy: %f" % avg_val_accuracy)

    # Loss über alle Batches
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    tqdm.write("Validation loss %f" % avg_val_loss)

    # Statistik speichern für Auswertung
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Validierung Loss': avg_val_loss,
            'Accuracy': avg_val_accuracy
        }
    )

In [ ]:
import pandas as pd

df_stats = pd.DataFrame(data=training_stats).set_index("epoch")
df_stats

In [ ]:
df_stats.plot()